In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [37]:
import os
os.environ["GROQ_API_KEY"] = ""

embedding_model = OllamaEmbeddings(model="nomic-embed-text")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=1.0,
    max_retries=2,
)

In [38]:
#Carregar pdf

pdf_link = "lotus.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [39]:
#Chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [40]:
# Salvar no Vector DB = Chroma

db = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory="text_index_2")


In [41]:
#Carregar DB

vectordb = Chroma(persist_directory="text_index_2", embedding_function=embedding_model)

#Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k":3})

#Construção da cadeia de promt para chamda do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [42]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
    return answer

In [44]:
user_question = input("User: ")
answer = ask(user_question)
print("Answer: ", answer)

Answer:  De acordo com o texto, TAG é considerado melhor que Text2SQL e RAG por várias razões:

1. **Flexibilidade**: TAG pode gerar mais do que apenas consultas SQL, podendo combinar múltiplas fontes e tipos de dados.
2. **Capacidade de lidar com dados de texto livre**: Text2SQL não pode lidar com dados de texto livre, enquanto TAG pode.
3. **Redução de "hallucinações"**: RAG pode fornecer respostas incompletas ou inaccuradas, especialmente quando não tem conhecimento do banco de dados ou não pode traduzir a intenção do usuário corretamente. TAG pareceaddressar esses problemas.
4. **Eficiência com grandes conjuntos de dados**: RAG é ineficiente com grandes conjuntos de dados, enquanto TAG parece ser mais eficiente.
5. **Capacidade de fornecer respostas abrangentes**: TAG pode fornecer respostas mais completas e abrangentes, especialmente em cenários que requerem razão sobre múltiplos pontos de dados.

No entanto, é importante notar que o texto não fornece uma comparação exaustiva entr